## This notebook loads post disaster data

In [89]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import rasterio
from shapely.geometry import Point, Polygon
from functools import reduce
import fhv

# UPAZILA SHAPEFILE
# ------------------------------------------------- #
shape = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm3_bbs_20180410.shp')
# Convert ADM3_PCODE of Mymensingh (45) division (total 378 unions) (45 -> 30)
f45t30 = '30' + shape.loc[shape['ADM1_PCODE'] == '45', 'ADM3_PCODE'].str[2:]
shape.loc[shape['ADM1_PCODE'] == '45', 'ADM3_PCODE'] = f45t30.values
shape['ADM3_PCODE'] = shape['ADM3_PCODE'].astype(int)
f45t30 = '30' + shape.loc[shape['ADM1_PCODE'] == '45', 'ADM2_PCODE'].str[2:]
shape.loc[shape['ADM1_PCODE'] == '45', 'ADM2_PCODE'] = f45t30.values
shape['ADM2_PCODE'] = shape['ADM2_PCODE'].astype(int)
ADM2 = shape[['ADM2_EN','ADM2_PCODE']].copy().drop_duplicates()
ADM2['ADM2_PCODE'] = ADM2['ADM2_PCODE'].astype(int)
if False:
    shape[['ADM2_PCODE','ADM2_EN','ADM3_PCODE','ADM3_EN']].sort_values(
        by='ADM3_PCODE').reset_index(drop=True).to_excel('./data/upazila_list.xlsx')
# ------------------------------------------------- #

# POPULATION DATA
# ------------------------------------------------- #
# BGD Census total population in 2011:  144,043,697
# BGD World Bank population in 2011:    149,273,778
# BGD World Bank population in 2017:    159,670,593
# ------------------------------------------------- #
df = fhv.LoadCensusBBS('./data/census2011/age 5 years group.xls')
popu2011 = df.sum(axis=1)
popu2017 = (popu2011/popu2011.sum()*159670593).astype(int)
popu2017_adm2 = popu2017.copy()
popu2017_adm2.index = (popu2017_adm2.index / 100).astype(int)
popu2017_adm2 = popu2017_adm2.groupby(popu2017_adm2.index).sum()
popu2017_adm2.index.name = 'ADM2_PCODE'; popu2017_adm2.name = 'Population'

### Disaster Damage and Loss data
The impacts (damage and loss) of 2017 August flood is obtained from Shelter Cluster DDM, MoDMR, NIRAPAD, etc.
- [Banladesh Monsoon Floods 2017, data table on Sep-3](https://www.sheltercluster.org/bangladesh-monsoon-floods-2017/documents/assessment-flood-damage-data-government-03092017)
- [Banladesh Monsoon Floods 2017, data table on Aug-30, from NIRAPAD Monthly Hazard Incident Report](https://www.nirapad.org.bd/home/resources/monthlyHazard)
- [72 hours Rapid Assessment Report NAWG V1](https://www.sheltercluster.org/bangladesh-monsoon-floods-2017/documents/nawg-72-hours-rapid-assessment-report-v1)

In [91]:
# DDM and NDRCC published in Sep-03-2017
df = pd.read_excel('./data/disaster_records/damagedata_DDM.xlsx', 
                   sheet_name='DDM, Sep 3',skiprows=1,skipfooter=1).drop('SL', axis=1).fillna(0)
df = df[['Name of affected Districts','No of Total  affecetd Families','No of Total  damaged Houses','No of Total   damaged Crops land (Hect)','No of Death People','No of Damaged Water point (Tube well)']]
df = df.rename(columns={'Name of affected Districts': 'ADM2_EN',
                       'No of Total  affecetd Families': 'Affected families',
                       'No of Total  damaged Houses': 'Affected houses',
                       'No of Total   damaged Crops land (Hect)': 'Affected crops land (Hect)',
                       'No of Death People': 'Death',
                       'No of Damaged Water point (Tube well)': 'Damaged tube well'})
# - Change the district names to be consistent with shapefile
df['ADM2_EN'] = df['ADM2_EN'].replace({'Rajshahi Dist': 'Rajshahi',
                                       'Moulvibazar': 'Maulvibazar',
                                       'Sunamjanj': 'Sunamganj',
                                       'Netrokona': 'Netrakona'})
# - Merge with ADM2 (Name and Code) just like join
df = pd.merge(df,ADM2,how='inner',left_on='ADM2_EN',right_on='ADM2_EN').drop('ADM2_EN',axis=1)
df0903 = df[df.columns[[-1,0,1,2,3,4]]]
# We will remove the Rajshahi district (ADM2_PCODE: 5081)
df0903 = df0903.loc[df0903['ADM2_PCODE'] != 5081].reset_index(drop=True)


# DDM and NDRCC published in Aug-30-2017
df = pd.read_excel('./data/disaster_records/damagedata_DDM.xlsx', 
                   sheet_name='DDM, Aug 30 (modified)',skiprows=0,skipfooter=1).fillna(0)
df = df.rename(columns={'Affected Districts': 'ADM2_EN',
                        'Affected People ( %. of total population)':'Affected population',
                        'No. of Damaged House':'Damaged houses',
                        'Affected Crops land (Hec.)':'Affected crops land (Hect)',
                        'No. of Death':'Death',
                        'No. of Displaced':'Displaced',
                        'No. of Affected Tube well':'Damaged tube well'})
# - Change the district names to be consistent with shapefile
df['ADM2_EN'] = df['ADM2_EN'].replace({'Brahmanbaria': 'Brahamanbaria',
                                       'Chadpur': 'Chandpur',
                                       'Moulvibazar': 'Maulvibazar',
                                       'Munsiganj': 'Munshiganj',
                                       'Netrokona': 'Netrakona',
                                       'Panchaghar': 'Panchagarh'})
# - Merge with ADM2 (Name and Code) just like join
df = pd.merge(df,ADM2,how='inner',left_on='ADM2_EN',right_on='ADM2_EN').drop('ADM2_EN',axis=1)
df0830 = df[df.columns[[-1,3,4,5,6,7,8,9,10,11,12,13]]]

# Merge to single DataFrame
temp = df0830[['ADM2_PCODE', 'Affected population', 'Affected Institution', 'Affected Road (km)', 'Displaced']]
damage = pd.merge(temp, df0903, how='inner', left_on='ADM2_PCODE', right_on='ADM2_PCODE')

### Post-flood health data
The post-flood health data is obtained from the [Directorate General of Health Services (DGHS)](https://dghs.gov.bd/index.php/en/home) - [Health Dashboard](http://103.247.238.81/webportal/pages/index.php).</br>
In the dashboard, the DGHS made a page for public health situation during/after the 2017 (July-August) flood in the Tableau format.
- By removing the duplicate rows, the numbers are similar to the [dashboards](http://103.247.238.81/webportal/pages/flood_affected.php).
- There are some differences from different periods of data
 
Here we use the following variables to represent the impacts on public health:
- Injuries Trauma Affected
- Diarrhoea Affected
- RTI, Drowning, snake bite, injury, eye disease, skin disease, and other cases

In [152]:
# Health records from Jul-01-2017 to Aug-15-2017 (Upazila scale)
cols = ['Division Code', 'Division Name',
        'District Code', 'District Name',
        'Upazila Code', 'Upazila Name',
        'Diarrhoea Affected', 'Diarrhoea Death',
        'Drowning Affected', 'Drowning Death',
        'Injuries Trauma Affected','Injuries Trauma Death',
        'Skin Disease Affected',
        'Snakebite Affected','Snakebite Death',
        'Is This Upazilla Currently Flood Affected',
#         'Latitude', 'Longitude',
        'Period']
df = pd.read_excel('./data/health_impact_2017Flood/dhis2_flood_affected (dbmis).xlsx',usecols=cols)[cols]
# - Convert Mymensingh (45) to Dhaka (30)
adm1_pcode = df['Division Code'].copy(); adm1_pcode.loc[adm1_pcode == 45] = 30
# - Assign ADM2_PCODE and ADM3_PCODE
df['ADM2_PCODE'] = adm1_pcode*10**2 + df['District Code']
assert np.isin(df['ADM2_PCODE'].unique(), shape.ADM2_PCODE).sum() == len(df['ADM2_PCODE'].unique())
df['ADM3_PCODE'] = adm1_pcode*10**4 + df['District Code']*10**2 + df['Upazila Code']
assert np.isin(df['ADM3_PCODE'].unique(), shape.ADM3_PCODE).sum() == len(df['ADM3_PCODE'].unique())
# - Reorder the DataFrame
df['Date'] = pd.DatetimeIndex(pd.to_datetime(df['Period'],format='%Y%m%d'))
df = df[['ADM3_PCODE','Date',*cols[6:-1]]]

In [153]:
df

,ADM3_PCODE,Date,Diarrhoea Affected,Diarrhoea Death,Drowning Affected,Drowning Death,Injuries Trauma Affected,Injuries Trauma Death,Skin Disease Affected,Snakebite Affected,Snakebite Death,Is This Upazilla Currently Flood Affected
0,100602,2017-07-01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100602,2017-07-01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100602,2017-07-01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100602,2017-07-01,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,100602,2017-07-01,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35753,609194,2017-08-15,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN
35754,609194,2017-08-15,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
35755,609194,2017-08-15,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
35756,609194,2017-08-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [110]:
temp = df.copy()
temp = temp.drop_duplicates() 
a = temp.groupby('ADM3_PCODE')[cols[6:-1]].sum()
a.sum()

Diarrhoea Affected                           19898.0
Diarrhoea Death                              80004.0
Drowning Affected                              250.0
Drowning Death                                 281.0
Injuries Trauma Affected                      4154.0
Injuries Trauma Death                            0.0
Skin Disease Affected                         4861.0
Snakebite Affected                             159.0
Snakebite Death                                 66.0
Is This Upazilla Currently Flood Affected     2620.0
dtype: float64

In [146]:
# Health data from Jul-23-2017 to Aug-26-2017 (District scale)
df = pd.read_excel('./data/health_impact_2017Flood/controlroom_flood (dbmis).xlsx')
df = df[['District',*df.columns[6:-1]]]
df = df.groupby('District').sum().reset_index()
df = df.rename(columns={'District':'ADM2_EN'})
df['ADM2_EN'] = df['ADM2_EN'].replace({'Brahmanbaria': 'Brahamanbaria'})
df = pd.merge(df,ADM2,how='inner',left_on='ADM2_EN',right_on='ADM2_EN').drop('ADM2_EN',axis=1)
df = df.set_index('ADM2_PCODE')

# - Diarrhea and Sum of other disease cases
col_other = ['No of RTI cases','No Of Eye Disease cases','No Of Drowning cases',
             'No Of Eye Disease Deaths','No Of Injury cases','No Of Injury Deaths',
             'No Of Snake Bite cases','No Of Snake Bite Deaths','No Of Skin Disease cases','No Of Other cases']
health_diarrhea = df['No Of Diarrhea Cases']
health_diarrhea.name = 'Diarrhea'
health_other = df[col_other].sum(1)
health_other.name = 'Other'
health2 = pd.merge(health_diarrhea,health_other, how='inner', left_index=True, right_index=True)